In [2]:
!pip install ktrain
!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

import ktrain
from ktrain import text
import numpy as np
import glob
import pandas as pd

  Cloning https://github.com/amaiya/eli5 (to revision tfkeras_0_10_1) to /tmp/pip-req-build-8kwxxkfl
  Running command git clone --filter=blob:none --quiet https://github.com/amaiya/eli5 /tmp/pip-req-build-8kwxxkfl
  Running command git checkout -b tfkeras_0_10_1 --track origin/tfkeras_0_10_1
  Switched to a new branch 'tfkeras_0_10_1'
  Branch 'tfkeras_0_10_1' set up to track remote branch 'tfkeras_0_10_1' from 'origin'.
  Resolved https://github.com/amaiya/eli5 to commit 72a5106bb9bcb8eb6a9b550ce3c0561eeb1fd2b3
  Preparing metadata (setup.py) ... done


In [3]:
trainDf = pd.read_csv('../input/sentnob-sentiment-analysis-in-noisy-bangla-texts/SentNoB Dataset/Train.csv')
valDf = pd.read_csv('../input/sentnob-sentiment-analysis-in-noisy-bangla-texts/SentNoB Dataset/Val.csv')

In [4]:
valDf

,Data,Label
0,আর আমার খুবেই আনন্দ লাকছে ভাইটি চাকরি পেয়েছে,1
1,ভাই আমাদের আগের মেয়র আনিচুল হক নাই যে আমাদের ক...,2
2,আমি মার্ক ওয়েন আর সনির বিশাল ভক্ত । একটা সময় ভ...,1
3,৩ মাস না যেতেই একেকজন ফুলে ফেপে আলুর দম,2
4,"বাংলাদেশের পুলিশ হলো নিরীহ মানুষের যম , আর অত্...",2
...,...,...
1562,আর বিচার হবে বলে মনে হয় না,2
1563,"ভাই রাতে গেলে পাবেন , ০১৮৬২২৮৪৪৯৫ অথবা ০১৮৬২৮০...",1
1564,আমাকে যে দিন ( যেই তারিখ এ ) ডক্টর এ্যাপয়েনমে...,1
1565,আরে ভাই এখান থেকে তো রাজনীতি করে বড় হচ্ছে আর য...,2


In [5]:
from sklearn.model_selection import train_test_split
train_body, test_body, train_cluster_id, test_cluster_id = trainDf['Data'].values, valDf['Data'].values, trainDf['Label'].values, valDf['Label'].values

In [6]:
#for Bengali

MODEL_NAME = 'csebuetnlp/banglabert'  # 
# csebuetnlp/banglabert
t = text.Transformer(MODEL_NAME, maxlen=200)


Downloading:   0%|          | 0.00/586 [00:00<?, ?B/s]

In [7]:
X_train = list(train_body)
y_train = list(train_cluster_id)
X_test = list(test_body)
y_test = list(test_cluster_id)

y_train = [str(x) for x in train_cluster_id]
y_test = [str(x) for x in test_cluster_id]

trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 15
	95percentile : 37
	99percentile : 47


Downloading:   0%|          | 0.00/119 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/528k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 37
	99percentile : 47


In [8]:
# learning_rate = 1e-4
learning_rate = 0.0001
epochs = 5

model = t.get_classifier()

learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=16)

learner.fit_onecycle(learning_rate, epochs)

404 Client Error: Not Found for url: https://huggingface.co/csebuetnlp/banglabert/resolve/main/tf_model.h5
/opt/conda/lib/python3.7/site-packages/ktrain/text/preprocessor.py:1172: UserWarning: Could not load a Tensorflow version of model. (If this worked before, it might be an out-of-memory issue.) Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  "Could not load a Tensorflow version of model. (If this worked before, it might be an out-of-memory issue.) "


Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]



begin training using onecycle policy with max lr of 0.0001...
Epoch 1/5
786/786 [==============================] - 351s 417ms/step - loss: 0.7874 - accuracy: 0.6563 - val_loss: 0.6680 - val_accuracy: 0.7320
Epoch 2/5
786/786 [==============================] - 327s 413ms/step - loss: 0.6137 - accuracy: 0.7494 - val_loss: 0.6333 - val_accuracy: 0.7256
Epoch 3/5
786/786 [==============================] - 327s 413ms/step - loss: 0.4992 - accuracy: 0.8068 - val_loss: 0.6379 - val_accuracy: 0.7250
Epoch 4/5
786/786 [==============================] - 327s 413ms/step - loss: 0.2864 - accuracy: 0.9000 - val_loss: 0.7842 - val_accuracy: 0.7511
Epoch 5/5
786/786 [==============================] - 328s 414ms/step - loss: 0.1205 - accuracy: 0.9626 - val_loss: 0.8676 - val_accuracy: 0.7626


In [9]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

           0       0.61      0.55      0.57       354
           1       0.79      0.83      0.81       623
           2       0.82      0.83      0.82       590

    accuracy                           0.76      1567
   macro avg       0.74      0.73      0.73      1567
weighted avg       0.76      0.76      0.76      1567



array([[193,  89,  72],
       [ 73, 514,  36],
       [ 53,  49, 488]])

In [10]:
predictor = ktrain.get_predictor(learner.model, preproc=t)
predictor.save('combined_model')

In [11]:
learner.model.summary()

Model: "tf_electra_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
electra (TFElectraMainLayer) multiple                  110026752 
_________________________________________________________________
classifier (TFElectraClassif multiple                  592899    
Total params: 110,619,651
Trainable params: 110,619,651
Non-trainable params: 0
_________________________________________________________________
